In [1]:
# 模型定义
import torch
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_net = nn.Sequential(
            # 1*28*28
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            # 1*14*14
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        )
        self.linear_net = nn.Sequential(
            # 1*7*7
            nn.Linear(64*7*7,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.conv_net(x)
        x = x.view(-1, 64*7*7)
        x = self.linear_net(x)
        return x

# 初始化网络（本次使用安装的torch版本仅使用CPU）
model = MyNet()

In [2]:
# 优化器定义
from torch.optim import Adam
optimizer = Adam(model.parameters(),lr=1e-3)

# 损失函数定义
import torch.nn.functional as F
loss_func = F.cross_entropy

In [3]:
# 定义训练过程
# 注意loss_func不一定通过传参形式给到trainer, 可以直接import
# device 是CPU或CUDA, 或者特定编号的GPU
def trainer(batch, model, optimizer, loss_func, device):
    # 将模型参数设为训练模式
    model.train()
    # 从batch中获取输入数据和标签(不一定有标签)
    x, y = batch
    # 将数据存入对应设备中
    x = x.to(device)
    y = y.to(device)
    # 梯度清零
    optimizer.zero_grad()
    # 前向传播
    y_hat = model(x)
    # 计算loss
    loss = loss_func(y_hat,y)
    # 反向传播获取梯度
    loss.backward()
    # 更新参数
    optimizer.step()

    # 计算准确率
    predictions = torch.argmax(y_hat, dim=1)  # 获取模型的预测结果
    correct_predictions = (predictions == y).sum().item()  # 统计正确的预测数量
    return loss.item() / y.shape[0], correct_predictions

In [4]:
def validate(val_loader, model, loss_func, device):
    # 将模型参数设为评估模式
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # 禁用梯度计算
        for batch in val_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)

            # 前向传播
            y_hat = model(x)
            # 计算loss
            loss = loss_func(y_hat, y)

            total_loss += loss.item() / y.size(0)

            # 计算准确率
            predictions = torch.argmax(y_hat, dim=1)
            correct_predictions += (predictions == y).sum().item()
            total_samples += y.size(0)

    # 计算平均损失和准确率
    average_loss = total_loss / len(val_loader)
    accuracy = correct_predictions / total_samples

    print(f"Validation Loss: {average_loss:.4f} | Accuracy: {accuracy * 100:.2f}%")

    return average_loss, accuracy

In [5]:
from torch.utils.data import Dataset
from torchvision.datasets import MNIST
import torchvision.transforms as T

# 定义数据集的转换
my_transform = T.Compose([
    T.ToTensor(),  # 将图像转换为张量
    T.Normalize((0.5,), (0.5,))  # 标准化张量，使其范围在[-1, 1]之间
])

# 初始化训练集和测试集
train_dataset = MNIST(root='./data', train=True, transform=my_transform, download=False)
test_dataset = MNIST(root='./data', train=False, transform=my_transform, download=False)

In [6]:
from torch.utils.data import DataLoader

# 定义 DataLoader 来加载数据
batch_size = 512
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

In [7]:
device = "cpu"
total_ep = 10
for ep in range(total_ep):
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        loss, correct_predictions = trainer(batch, model, optimizer, loss_func, device)
        total_loss += loss
        total_correct += correct_predictions
        total_samples += batch[1].shape[0]
    average_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_samples
    print(f"Epoch: {ep} Training Loss: {average_loss:.4f} | Accuracy: {accuracy * 100:.2f}%")
        
    if (ep+1) % 2 == 0:
        validate(test_loader, model, loss_func, device)

Epoch: 0 Training Loss: 0.0007 | Accuracy: 89.03%
Epoch: 1 Training Loss: 0.0001 | Accuracy: 98.19%
Validation Loss: 0.0001 | Accuracy: 98.51%
Epoch: 2 Training Loss: 0.0001 | Accuracy: 98.76%
Epoch: 3 Training Loss: 0.0001 | Accuracy: 99.11%
Validation Loss: 0.0001 | Accuracy: 98.83%
Epoch: 4 Training Loss: 0.0000 | Accuracy: 99.25%
Epoch: 5 Training Loss: 0.0000 | Accuracy: 99.50%
Validation Loss: 0.0001 | Accuracy: 98.94%
Epoch: 6 Training Loss: 0.0000 | Accuracy: 99.46%
Epoch: 7 Training Loss: 0.0000 | Accuracy: 99.63%
Validation Loss: 0.0001 | Accuracy: 99.10%
Epoch: 8 Training Loss: 0.0000 | Accuracy: 99.72%
Epoch: 9 Training Loss: 0.0000 | Accuracy: 99.76%
Validation Loss: 0.0001 | Accuracy: 99.11%
